In [ ]:
import pandas as pd
import torch
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F

# GCN Layer Implementation
class GCNLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super(GCNLayer, self).__init__()
        self.linear = nn.Linear(in_features, out_features)
        
    def forward(self, x, adj):
        # Normalize adjacency matrix
        deg = torch.sum(adj, dim=1)
        deg_inv_sqrt = torch.pow(deg, -0.5)
        deg_inv_sqrt[torch.isinf(deg_inv_sqrt)] = 0
        norm_adj = torch.diag(deg_inv_sqrt) @ adj @ torch.diag(deg_inv_sqrt)
        
        # GCN propagation rule
        support = self.linear(x)
        output = torch.matmul(norm_adj, support)
        return output

# Enhanced Generator with GCN layers
class GCNGenerator(nn.Module):
    def __init__(self, latent_size, hidden_size, node_features, num_nodes):
        super(GCNGenerator, self).__init__()
        
        self.num_nodes = num_nodes
        self.node_features = node_features
        
        # Initial linear layers for latent vector
        self.fc1 = nn.Linear(latent_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, num_nodes * hidden_size)
        
        # GCN layers
        self.gcn1 = GCNLayer(hidden_size, hidden_size)
        self.gcn2 = GCNLayer(hidden_size, hidden_size)
        self.gcn3 = GCNLayer(hidden_size, node_features)
        
        # Batch normalization layers
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.bn2 = nn.BatchNorm1d(hidden_size)
        
    def forward(self, z, adj):
        # Initial shape transformation
        x = F.relu(self.fc1(z))
        x = F.relu(self.fc2(x))
        x = x.view(-1, self.num_nodes, x.size(1) // self.num_nodes)
        
        # Apply GCN layers with residual connections
        x1 = F.relu(self.bn1(self.gcn1(x, adj).transpose(1, 2)).transpose(1, 2))
        x2 = F.relu(self.bn2(self.gcn2(x1, adj).transpose(1, 2)).transpose(1, 2))
        x3 = self.gcn3(x2, adj)
        
        return torch.sigmoid(x3)

# Enhanced Discriminator with GCN layers
class GCNDiscriminator(nn.Module):
    def __init__(self, node_features, hidden_size, num_nodes):
        super(GCNDiscriminator, self).__init__()
        
        # GCN layers
        self.gcn1 = GCNLayer(node_features, hidden_size)
        self.gcn2 = GCNLayer(hidden_size, hidden_size)
        
        # Final classification layers
        self.fc1 = nn.Linear(hidden_size * num_nodes, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)
        
        # Batch normalization layers
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.bn2 = nn.BatchNorm1d(hidden_size)
        
    def forward(self, x, adj):
        # Apply GCN layers
        x1 = F.relu(self.bn1(self.gcn1(x, adj).transpose(1, 2)).transpose(1, 2))
        x2 = F.relu(self.bn2(self.gcn2(x1, adj).transpose(1, 2)).transpose(1, 2))
        
        # Flatten and apply final layers
        x3 = x2.view(x2.size(0), -1)
        x4 = F.relu(self.fc1(x3))
        return torch.sigmoid(self.fc2(x4))

# Data preprocessing functions remain the same as in your code
def df_label_encoder(df, columns):
    le = preprocessing.LabelEncoder()
    for col in columns:
        df[col] = le.fit_transform(df[col].astype(str))
    return df

def preprocess(df):
    df = df_label_encoder(df, ['nameOrig', 'nameDest', 'type'])
    df['amount'] = (df['amount'] - df['amount'].min()) / (df['amount'].max() - df['amount'].min())
    df['node_from'] = df['nameOrig'].astype(str)
    df['node_to'] = df['nameDest'].astype(str)
    df = df.sort_values(by=['node_from'])
    node_list = pd.concat([df['node_from'], df['node_to']]).unique()
    return df, node_list

def create_graph_data(df, node_list):
    node_map = {node: idx for idx, node in enumerate(node_list)}
    
    # Create edge index
    edge_index = np.array([
        [node_map[from_node], node_map[to_node]] 
        for from_node, to_node in zip(df['node_from'], df['node_to'])
    ], dtype=np.int64).T
    
    # Create adjacency matrix
    num_nodes = len(node_list)
    adj = torch.zeros((num_nodes, num_nodes))
    adj[edge_index[0], edge_index[1]] = 1
    
    # Make adjacency matrix symmetric (undirected graph)
    adj = adj + adj.t()
    adj[adj > 1] = 1
    
    # Add self-loops
    adj = adj + torch.eye(num_nodes)
    
    # Create node features
    node_features = torch.tensor(df[['amount', 'type']].values, dtype=torch.float)
    
    # Labels
    labels = torch.tensor(df['isFraud'].values, dtype=torch.long)
    
    return node_features, adj, labels

# Training function
def train_gcn_gan(generator, discriminator, node_features, adj, labels, num_epochs=100):
    # Initialize optimizers
    optimizer_g = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
    optimizer_d = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
    criterion = nn.BCELoss()
    
    # Training statistics
    g_losses = []
    d_losses = []
    
    num_nodes = node_features.size(0)
    latent_size = 64
    batch_size = 32
    
    for epoch in range(num_epochs):
        # Train Discriminator
        optimizer_d.zero_grad()
        
        # Real data
        real_labels = torch.ones(batch_size, 1)
        d_real = discriminator(node_features.unsqueeze(0).expand(batch_size, -1, -1), adj)
        d_real_loss = criterion(d_real, real_labels)
        
        # Fake data
        z = torch.randn(batch_size, latent_size)
        fake_features = generator(z, adj)
        fake_labels = torch.zeros(batch_size, 1)
        d_fake = discriminator(fake_features, adj)
        d_fake_loss = criterion(d_fake, fake_labels)
        
        # Combined discriminator loss
        d_loss = d_real_loss + d_fake_loss
        d_loss.backward()
        optimizer_d.step()
        
        # Train Generator
        optimizer_g.zero_grad()
        
        z = torch.randn(batch_size, latent_size)
        fake_features = generator(z, adj)
        g_fake = discriminator(fake_features, adj)
        g_loss = criterion(g_fake, real_labels)
        
        g_loss.backward()
        optimizer_g.step()
        
        # Record losses
        g_losses.append(g_loss.item())
        d_losses.append(d_loss.item())
        
        if epoch % 10 == 0:
            print(f'Epoch [{epoch}/{num_epochs}], D Loss: {d_loss.item():.4f}, G Loss: {g_loss.item():.4f}')
    
    return g_losses, d_losses

# Main execution
if __name__ == "__main__":
    # Load and preprocess data
    df = pd.read_csv('paysim/paysim.csv')
    df = df.sample(frac=0.2, random_state=42).reset_index(drop=True)
    df, node_list = preprocess(df)
    node_features, adj, labels = create_graph_data(df, node_list)
    
    # Initialize models
    num_nodes = len(node_list)
    node_feature_size = node_features.size(1)
    hidden_size = 128
    latent_size = 64
    
    generator = GCNGenerator(latent_size, hidden_size, node_feature_size, num_nodes)
    discriminator = GCNDiscriminator(node_feature_size, hidden_size, num_nodes)
    
    # Train the model
    g_losses, d_losses = train_gcn_gan(generator, discriminator, node_features, adj, labels)
    
    # Plot training losses
    plt.figure(figsize=(10, 5))
    plt.plot(g_losses, label='Generator Loss')
    plt.plot(d_losses, label='Discriminator Loss')
    plt.xlabel('Iteration')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('GCN-GAN Training Losses')
    plt.show()
    
    # Generate synthetic samples
    with torch.no_grad():
        z = torch.randn(32, latent_size)
        synthetic_features = generator(z, adj)
        
    print("Synthetic features shape:", synthetic_features.shape)